In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Reshape, Flatten, Input
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import layers
from tensorflow.keras import layers, models
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2


In [2]:
def load_images(data_path, image_size):
    images = []
    for img_file in os.listdir(data_path):
        try:
            img_path = os.path.join(data_path, img_file)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (image_size[1], image_size[0]))
            img = (img - 127.5) / 127.5 
            img = np.expand_dims(img, axis=-1)
            images.append(img)
        except Exception as e:
            print(f"Error loading image {img_file}: {e}")

    if len(images) == 0:
        print("No images loaded or encountered an error while loading images.")
    else:
        print(f"Total images loaded: {len(images)}")
        print(f"Shape of the first loaded image: {images[0].shape}")
    
    return np.array(images)

In [3]:
# Define the generator model
def build_generator(z_dim):
    model = tf.keras.Sequential()
    model.add(layers.Dense(256, input_shape=(100,)))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dense(512))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dense(948 * 948, activation='tanh'))
    model.add(layers.Reshape((948, 948, 1)))
    return model

# Define the discriminator model
def build_discriminator(image_shape):
    model = tf.keras.Sequential()
    model.add(layers.Flatten(input_shape=(948, 948, 1)))
    model.add(layers.Dense(512))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dense(256))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model


In [4]:
from tensorflow.keras.layers import Input

# 假設圖像大小為 64x64 的灰度圖（只有一個通道）
img_rows = 948
img_cols = 948
channels = 1
img_shape = (img_rows, img_cols, channels)

z_dim = 100

# 創建鉴别器和生成器
discriminator = build_discriminator(img_shape)
discriminator.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

generator = build_generator(z_dim)

# 創建 GAN
z = Input(shape=(z_dim,))
img = generator(z)

# 將鉴别器設置為不可訓練
discriminator.trainable = False

# 連接鉴别器和生成器
validity = discriminator(img)
gan = Model(z, validity)
gan.compile(loss='binary_crossentropy', optimizer='adam')

In [5]:
def sample_images(epoch, save_path='generated_images'):
    os.makedirs(save_path, exist_ok=True)
    noise = np.random.normal(0, 1, (1, z_dim))
    gen_imgs = generator.predict(noise)
    gen_imgs = 0.5 * gen_imgs + 0.5  

    file_path = os.path.join(save_path, f"generated_{epoch}.jpg")
    plt.imshow(gen_imgs[0, :, :, 0], cmap='gray')
    plt.axis('off')
    plt.savefig(file_path)
    plt.close()

# 训练 GAN
def train(epochs, batch_size, data_path, img_shape):
    X_train = load_images(data_path, img_shape)
    real = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for epoch in range(epochs):
        # 选择真实图像的随机批次
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        imgs = X_train[idx]

        # 生成假图像
        noise = np.random.normal(0, 1, (batch_size, z_dim))
        gen_imgs = generator.predict(noise)

        # 训练鉴别器
        d_loss_real = discriminator.train_on_batch(imgs, real)
        d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # 训练生成器
        noise = np.random.normal(0, 1, (batch_size, z_dim))
        g_loss = gan.train_on_batch(noise, real)

        # 打印进度
        print(f"Epoch {epoch}/{epochs} [D loss: {d_loss[0]} | D accuracy: {100 * d_loss[1]}] [G loss: {g_loss}]")

        # 保存生成的图像
        if epoch % 1000 == 0:
            sample_images(epoch)

In [6]:

data_path = "C:\\Users\\child\\Desktop\\pic"


train(5000, 64, data_path, img_shape)

Total images loaded: 6
Shape of the first loaded image: (948, 948, 1)
2/2 [==============================] - 0s 117ms/step


Epoch 0/5000 [D loss: 0.7534920275211334 | D accuracy: 5.46875] [G loss: 1.7068895101547241]
2/2 [==============================] - 0s 112ms/step
Epoch 1/5000 [D loss: 631.4595947265625 | D accuracy: 50.0] [G loss: 0.0]
2/2 [==============================] - 0s 117ms/step
Epoch 2/5000 [D loss: 533.54833984375 | D accuracy: 50.0] [G loss: 0.0]
2/2 [==============================] - 0s 117ms/step
Epoch 3/5000 [D loss: 0.0 | D accuracy: 100.0] [G loss: 1276.9207763671875]
2/2 [==============================] - 0s 118ms/step
Epoch 4/5000 [D loss: 3458.302001953125 | D accuracy: 50.0] [G loss: 0.0]
2/2 [==============================] - 0s 114ms/step
Epoch 5/5000 [D loss: 263.8016357421875 | D accuracy: 50.0] [G loss: 0.0]
2/2 [==============================] - 0s 117ms/step
Epoch 6/5000 [D loss: 348.0960388183594 | D accuracy: 50.0] [G loss: 0.0]
2/2 [===============

2/2 [==============================] - 0s 117ms/step
Epoch 56/5000 [D loss: 388.57244873046875 | D accuracy: 50.0] [G loss: 161.1827392578125]
2/2 [==============================] - 0s 116ms/step
Epoch 57/5000 [D loss: 117.66157531738281 | D accuracy: 53.90625] [G loss: 514.4370727539062]
2/2 [==============================] - 0s 116ms/step
Epoch 58/5000 [D loss: 0.0 | D accuracy: 100.0] [G loss: 1515.925048828125]
2/2 [==============================] - 0s 112ms/step
Epoch 59/5000 [D loss: 0.0 | D accuracy: 100.0] [G loss: 1804.645263671875]
2/2 [==============================] - 0s 115ms/step
Epoch 60/5000 [D loss: 3.3818371295928955 | D accuracy: 98.4375] [G loss: 1339.30615234375]
2/2 [==============================] - 0s 113ms/step
Epoch 61/5000 [D loss: 89.77528381347656 | D accuracy: 72.65625] [G loss: 1673.4993896484375]
2/2 [==============================] - 0s 113ms/step
Epoch 62/5000 [D loss: 0.9597046971321106 | D accuracy: 99.21875] [G loss: 2830.50830078125]
2/2 [=========

2/2 [==============================] - 0s 115ms/step
Epoch 174/5000 [D loss: 28.085399627685547 | D accuracy: 97.65625] [G loss: 9423.27734375]
2/2 [==============================] - 0s 116ms/step
Epoch 175/5000 [D loss: 47.752647399902344 | D accuracy: 96.875] [G loss: 11849.052734375]
2/2 [==============================] - 0s 115ms/step
Epoch 176/5000 [D loss: 198.79953002929688 | D accuracy: 49.21875] [G loss: 9434.3564453125]
2/2 [==============================] - 0s 116ms/step
Epoch 177/5000 [D loss: 51.25029754638672 | D accuracy: 97.65625] [G loss: 7018.59228515625]
2/2 [==============================] - 0s 118ms/step
Epoch 178/5000 [D loss: 63.7972412109375 | D accuracy: 97.65625] [G loss: 5549.33642578125]
2/2 [==============================] - 0s 116ms/step
Epoch 179/5000 [D loss: 21.185726165771484 | D accuracy: 97.65625] [G loss: 4452.2373046875]
2/2 [==============================] - 0s 116ms/step
Epoch 180/5000 [D loss: 153.43603515625 | D accuracy: 93.75] [G loss: 3590.1

2/2 [==============================] - 0s 113ms/step
Epoch 288/5000 [D loss: 0.0 | D accuracy: 100.0] [G loss: 3451.25146484375]
2/2 [==============================] - 0s 113ms/step
Epoch 289/5000 [D loss: 117.2146472632885 | D accuracy: 49.21875] [G loss: 2047.4505615234375]
2/2 [==============================] - 0s 116ms/step
Epoch 290/5000 [D loss: 43.999794006347656 | D accuracy: 90.625] [G loss: 1283.7125244140625]
2/2 [==============================] - 0s 110ms/step
Epoch 291/5000 [D loss: 49.89381790161133 | D accuracy: 90.625] [G loss: 947.473876953125]
2/2 [==============================] - 0s 115ms/step
Epoch 292/5000 [D loss: 57.40406036376953 | D accuracy: 87.5] [G loss: 996.8699340820312]
2/2 [==============================] - 0s 110ms/step
Epoch 293/5000 [D loss: 76.32977294921875 | D accuracy: 89.0625] [G loss: 1038.2779541015625]
2/2 [==============================] - 0s 110ms/step
Epoch 294/5000 [D loss: 61.3492431640625 | D accuracy: 85.9375] [G loss: 1529.72924804687

2/2 [==============================] - 0s 110ms/step
Epoch 402/5000 [D loss: 0.0 | D accuracy: 100.0] [G loss: 1564.3333740234375]
2/2 [==============================] - 0s 112ms/step
Epoch 403/5000 [D loss: 101.2953109741211 | D accuracy: 50.0] [G loss: 1029.252197265625]
2/2 [==============================] - 0s 109ms/step
Epoch 404/5000 [D loss: 1.0974069238328639e-38 | D accuracy: 100.0] [G loss: 648.918701171875]
2/2 [==============================] - 0s 110ms/step
Epoch 405/5000 [D loss: 10.568138122558594 | D accuracy: 87.5] [G loss: 488.5543518066406]
2/2 [==============================] - 0s 121ms/step
Epoch 406/5000 [D loss: 6.349635124206543 | D accuracy: 89.0625] [G loss: 548.0296630859375]
2/2 [==============================] - 0s 112ms/step
Epoch 407/5000 [D loss: 0.2301839292049408 | D accuracy: 99.21875] [G loss: 650.2509765625]
2/2 [==============================] - 0s 109ms/step
Epoch 408/5000 [D loss: 0.0 | D accuracy: 100.0] [G loss: 727.5457763671875]
2/2 [========

...

2/2 [==============================] - 0s 112ms/step
Epoch 4676/5000 [D loss: 0.40676816552877426 | D accuracy: 70.3125] [G loss: 0.9924466013908386]
2/2 [==============================] - 0s 115ms/step
Epoch 4677/5000 [D loss: 0.24217846989631653 | D accuracy: 100.0] [G loss: 1.8297648429870605]
2/2 [==============================] - 0s 114ms/step
Epoch 4678/5000 [D loss: 0.4721710979938507 | D accuracy: 100.0] [G loss: 1.1193931102752686]
2/2 [==============================] - 0s 113ms/step
Epoch 4679/5000 [D loss: 0.33502401411533356 | D accuracy: 90.625] [G loss: 1.173728585243225]
2/2 [==============================] - 0s 111ms/step
Epoch 4680/5000 [D loss: 0.22015821933746338 | D accuracy: 98.4375] [G loss: 1.9668124914169312]
2/2 [==============================] - 0s 116ms/step
Epoch 4681/5000 [D loss: 0.3856189548969269 | D accuracy: 100.0] [G loss: 1.2706537246704102]
2/2 [==============================] - 0s 114ms/step
Epoch 4682/5000 [D loss: 0.28945691883563995 | D accuracy

2/2 [==============================] - 0s 115ms/step
Epoch 4786/5000 [D loss: 2.8653951287269592 | D accuracy: 1.5625] [G loss: 0.05722189322113991]
2/2 [==============================] - 0s 116ms/step
Epoch 4787/5000 [D loss: 2.2190985847555567 | D accuracy: 50.0] [G loss: 0.15672793984413147]
2/2 [==============================] - 0s 126ms/step
Epoch 4788/5000 [D loss: 0.26590141479391605 | D accuracy: 93.75] [G loss: 4.013607025146484]
2/2 [==============================] - 0s 119ms/step
Epoch 4789/5000 [D loss: 0.8066490441560745 | D accuracy: 50.0] [G loss: 1.2932493686676025]
2/2 [==============================] - 0s 117ms/step
Epoch 4790/5000 [D loss: 0.39360741525888443 | D accuracy: 70.3125] [G loss: 1.7777721881866455]
2/2 [==============================] - 0s 120ms/step
Epoch 4791/5000 [D loss: 0.0342454481869936 | D accuracy: 100.0] [G loss: 4.838578224182129]
2/2 [==============================] - 0s 117ms/step
Epoch 4792/5000 [D loss: 0.6938013732433319 | D accuracy: 44.5

2/2 [==============================] - 0s 115ms/step
Epoch 4842/5000 [D loss: 2.961044430732727 | D accuracy: 5.46875] [G loss: 0.11557984352111816]
2/2 [==============================] - 0s 115ms/step
Epoch 4843/5000 [D loss: 1.7323851305554854 | D accuracy: 50.0] [G loss: 0.7639903426170349]
2/2 [==============================] - 0s 117ms/step
Epoch 4844/5000 [D loss: 0.0488103274255991 | D accuracy: 100.0] [G loss: 5.393357276916504]
2/2 [==============================] - 0s 116ms/step
Epoch 4845/5000 [D loss: 2.683193638920784 | D accuracy: 45.3125] [G loss: 0.2581254243850708]
2/2 [==============================] - 0s 120ms/step
Epoch 4846/5000 [D loss: 1.5389124238863587 | D accuracy: 50.0] [G loss: 0.2987577021121979]
2/2 [==============================] - 0s 118ms/step
Epoch 4847/5000 [D loss: 0.29765655822120607 | D accuracy: 88.28125] [G loss: 2.966858386993408]
2/2 [==============================] - 0s 118ms/step
Epoch 4848/5000 [D loss: 1.2434509992599487 | D accuracy: 36.7

Epoch 4897/5000 [D loss: 0.15535515174269676 | D accuracy: 99.21875] [G loss: 2.613630771636963]
2/2 [==============================] - 0s 111ms/step
Epoch 4898/5000 [D loss: 0.21183837950229645 | D accuracy: 100.0] [G loss: 1.770836591720581]
2/2 [==============================] - 0s 119ms/step
Epoch 4899/5000 [D loss: 0.1738959141075611 | D accuracy: 96.875] [G loss: 2.887140989303589]
2/2 [==============================] - 0s 114ms/step
Epoch 4900/5000 [D loss: 0.278390496969223 | D accuracy: 96.875] [G loss: 2.241042137145996]
2/2 [==============================] - 0s 112ms/step
Epoch 4901/5000 [D loss: 0.19201012700796127 | D accuracy: 97.65625] [G loss: 2.852560520172119]
2/2 [==============================] - 0s 120ms/step
Epoch 4902/5000 [D loss: 0.5231597721576691 | D accuracy: 73.4375] [G loss: 1.944592833518982]
2/2 [==============================] - 0s 112ms/step
Epoch 4903/5000 [D loss: 0.2242790013551712 | D accuracy: 98.4375] [G loss: 2.8957128524780273]
2/2 [===========

2/2 [==============================] - 0s 112ms/step
Epoch 4953/5000 [D loss: 14.496953964234281 | D accuracy: 50.0] [G loss: 0.6842005848884583]
2/2 [==============================] - 0s 112ms/step
Epoch 4954/5000 [D loss: 8.709616075479668e-06 | D accuracy: 100.0] [G loss: 58.90980529785156]
2/2 [==============================] - 0s 113ms/step
Epoch 4955/5000 [D loss: 45.99351358413696 | D accuracy: 0.0] [G loss: 3.515425078148837e-06]
2/2 [==============================] - 0s 115ms/step
Epoch 4956/5000 [D loss: 8.019756317138674 | D accuracy: 50.0] [G loss: 32.719093322753906]
2/2 [==============================] - 0s 112ms/step
Epoch 4957/5000 [D loss: 22.226847052574158 | D accuracy: 0.78125] [G loss: 5.56118106842041]
2/2 [==============================] - 0s 116ms/step
Epoch 4958/5000 [D loss: 0.0026649847295630025 | D accuracy: 100.0] [G loss: 22.47677230834961]
2/2 [==============================] - 0s 110ms/step
Epoch 4959/5000 [D loss: 34.08084487915039 | D accuracy: 0.0] [G

In [7]:
# 呼叫 sample_images 函式並儲存生成的圖像
sample_images(50,data_path)

1/1 [==============================] - 0s 84ms/step


In [8]:


# 儲存整個生成器模型（包含結構和權重）
generator.save('full_generator_model.h5')

C:\Users\child\anaconda3\envs\tes1219\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [9]:
sample_images(1000,data_path)

1/1 [==============================] - 0s 83ms/step
